In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [8]:
versions = ['intens', 'zen', 'life']
annees = [2012, 2013, 2014]

In [9]:
data_lbc = pd.DataFrame(columns=['Version', 'Année', 'Kilométrage', 'Prix', 'Téléphone', 'Pro'])
data_argus = pd.DataFrame(columns=['Année', 'Version', 'Cote'])

In [10]:
def retrieve_phone_number_desc(soup):
    desc = soup.find('div', {'itemprop': 'description'})
    phone_numbers = re.findall(r'((?:\d[\s\.-]?){10})', desc.get_text())
    if len(phone_numbers) > 0:
        return re.sub(r'\D', '', phone_numbers[0])
    else:
        return None
    
    
def retrieve_version(soup):
    titre = soup.find('h1', {'itemprop': 'name'}).get_text().lower()
    version = None
    for v in versions:
        if v in titre:
            version = v
            break
    return version


def retrieve_year(soup):
    annee = soup.find('td', {'itemprop': 'releaseDate'})
    if annee is not None:
        return int(re.sub(r'\D', '', annee.get_text()))
    else:
        return None


def retrieve_kilometers(soup):
    kilometrage = None
    for c in soup.find('div', {'class': 'lbcParams criterias'}).findAll('tr'):
        if u'Kilométrage' in c.find('th').get_text():
            kilometrage = int(re.sub(r'\D', '', c.find('td').get_text()))
            break
    return kilometrage
    
def retrieve_price(soup):
    prix = soup.find('span', {'class': 'price'})
    if prix is not None:
        return float(re.sub(r'\D', '', prix.get_text()))
    else:
        return None

def retrieve_pro(soup):
    pro = soup.find('span', {'class': 'ad_pro'})
    return pro is not None

In [11]:
urls_lbc = [
    "http://www.leboncoin.fr/voitures/offres/ile_de_france/?f=a&th=1&fu=4&q=zoe&it=1",
    "http://www.leboncoin.fr/voitures/offres/provence_alpes_cote_d_azur/?f=a&th=1&fu=4&q=zoe&it=1",
    "http://www.leboncoin.fr/voitures/offres/aquitaine/?f=a&th=1&fu=4&q=zoe&it=1"
]

for url in urls_lbc:
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    ads = soup.findAll('div', {'class': 'lbc'})
    for ad in ads:
        ad_url = ad.parent['href']
        print 'Annonce: ' + ad_url
        r = requests.get(ad_url)
        soup = BeautifulSoup(r.text, 'html.parser')
        data_lbc = data_lbc.append({
            'Version': retrieve_version(soup),
            'Année': retrieve_year(soup),
            'Kilométrage': retrieve_kilometers(soup),
            'Prix': retrieve_price(soup),
            'Téléphone': retrieve_phone_number_desc(soup),
            'Pro': retrieve_pro(soup)
        }, ignore_index=True)

Annonce: http://www.leboncoin.fr/voitures/827471549.htm?ca=12_s
Annonce: http://www.leboncoin.fr/voitures/890724432.htm?ca=12_s
Annonce: http://www.leboncoin.fr/voitures/809213291.htm?ca=12_s
Annonce: http://www.leboncoin.fr/voitures/889029891.htm?ca=12_s
Annonce: http://www.leboncoin.fr/voitures/856067960.htm?ca=12_s
Annonce: http://www.leboncoin.fr/voitures/798949165.htm?ca=12_s
Annonce: http://www.leboncoin.fr/voitures/814195720.htm?ca=12_s
Annonce: http://www.leboncoin.fr/voitures/878940912.htm?ca=12_s
Annonce: http://www.leboncoin.fr/voitures/878941074.htm?ca=12_s
Annonce: http://www.leboncoin.fr/voitures/835269788.htm?ca=12_s
Annonce: http://www.leboncoin.fr/voitures/815020992.htm?ca=12_s
Annonce: http://www.leboncoin.fr/voitures/715971245.htm?ca=12_s
Annonce: http://www.leboncoin.fr/voitures/848861858.htm?ca=12_s
Annonce: http://www.leboncoin.fr/voitures/871384960.htm?ca=12_s
Annonce: http://www.leboncoin.fr/voitures/892587949.htm?ca=21_s
Annonce: http://www.leboncoin.fr/voiture